In [1]:
# %matplotlib widget

from __future__ import annotations

import re
from collections import defaultdict
from dataclasses import dataclass, field
from itertools import permutations, product
from math import inf
from random import choice

import matplotlib.colors as mcolors
import matplotlib.pyplot as plt
import networkx as nx
import numpy as np
import numpy.typing as npt
from mpl_toolkits.mplot3d import axes3d
from numpy import int_, object_
from numpy.typing import NDArray
from test_utilities import test
from util import print_hex

COLORS = list(mcolors.CSS4_COLORS.keys())

<link href="style.css" rel="stylesheet"></link>

<article class="day-desc read-aloud"><h2>--- Day 23: Amphipod ---</h2><p>A group of <a href="https://en.wikipedia.org/wiki/Amphipoda" target="_blank">amphipods</a> notice your fancy submarine and flag you down. "With such an impressive shell," one amphipod <span title="What? You didn't know amphipods can talk?">says</span>, "surely you can help us with a question that has stumped our best scientists."</p>
<p>They go on to explain that a group of timid, stubborn amphipods live in a nearby burrow. Four types of amphipods live there: <em>Amber</em> (<code>A</code>), <em>Bronze</em> (<code>B</code>), <em>Copper</em> (<code>C</code>), and <em>Desert</em> (<code>D</code>). They live in a burrow that consists of a <em>hallway</em> and four <em>side rooms</em>. The side rooms are initially full of amphipods, and the hallway is initially empty.</p>
<p>They give you a <em>diagram of the situation</em> (your puzzle input), including locations of each amphipod (<code>A</code>, <code>B</code>, <code>C</code>, or <code>D</code>, each of which is occupying an otherwise open space), walls (<code>#</code>), and open space (<code>.</code>).</p>
<p>For example:</p>
<pre><code>#############
#...........#
###B#C#B#D###
  #A#D#C#A#
  #########
</code></pre>
<p>The amphipods would like a method to organize every amphipod into side rooms so that each side room contains one type of amphipod and the types are sorted <code>A</code>-<code>D</code> going left to right, like this:</p>
<pre><code>#############
#...........#
###A#B#C#D###
  #A#B#C#D#
  #########
</code></pre>
<p>Amphipods can move up, down, left, or right so long as they are moving into an unoccupied open space. Each type of amphipod requires a different amount of <em>energy</em> to move one step: Amber amphipods require <code>1</code> energy per step, Bronze amphipods require <code>10</code> energy, Copper amphipods require <code>100</code>, and Desert ones require <code>1000</code>. The amphipods would like you to find a way to organize the amphipods that requires the <em>least total energy</em>.</p>
<p>However, because they are timid and stubborn, the amphipods have some extra rules:</p>
<ul>
<li>Amphipods will never <em>stop on the space immediately outside any room</em>. They can move into that space so long as they immediately continue moving. (Specifically, this refers to the four open spaces in the hallway that are directly above an amphipod starting position.)</li>
<li>Amphipods will never <em>move from the hallway into a room</em> unless that room is their destination room <em>and</em> that room contains no amphipods which do not also have that room as their own destination. If an amphipod's starting room is not its destination room, it can stay in that room until it leaves the room. (For example, an Amber amphipod will not move from the hallway into the right three rooms, and will only move into the leftmost room if that room is empty or if it only contains other Amber amphipods.)</li>
<li>Once an amphipod stops moving in the hallway, <em>it will stay in that spot until it can move into a room</em>. (That is, once any amphipod starts moving, any other amphipods currently in the hallway are locked in place and will not move again until they can move fully into a room.)</li>
</ul>
<p>In the above example, the amphipods can be organized using a minimum of <code><em>12521</em></code> energy. One way to do this is shown below.</p>
<p>Starting configuration:</p>
<pre><code>#############
#...........#
###B#C#B#D###
  #A#D#C#A#
  #########
</code></pre>
<p>One Bronze amphipod moves into the hallway, taking 4 steps and using <code>40</code> energy:</p>
<pre><code>#############
#...B.......#
###B#C#.#D###
  #A#D#C#A#
  #########
</code></pre>
<p>The only Copper amphipod not in its side room moves there, taking 4 steps and using <code>400</code> energy:</p>
<pre><code>#############
#...B.......#
###B#.#C#D###
  #A#D#C#A#
  #########
</code></pre>
<p>A Desert amphipod moves out of the way, taking 3 steps and using <code>3000</code> energy, and then the Bronze amphipod takes its place, taking 3 steps and using <code>30</code> energy:</p>
<pre><code>#############
#.....D.....#
###B#.#C#D###
  #A#B#C#A#
  #########
</code></pre>
<p>The leftmost Bronze amphipod moves to its room using <code>40</code> energy:</p>
<pre><code>#############
#.....D.....#
###.#B#C#D###
  #A#B#C#A#
  #########
</code></pre>
<p>Both amphipods in the rightmost room move into the hallway, using <code>2003</code> energy in total:</p>
<pre><code>#############
#.....D.D.A.#
###.#B#C#.###
  #A#B#C#.#
  #########
</code></pre>
<p>Both Desert amphipods move into the rightmost room using <code>7000</code> energy:</p>
<pre><code>#############
#.........A.#
###.#B#C#D###
  #A#B#C#D#
  #########
</code></pre>
<p>Finally, the last Amber amphipod moves into its room, using <code>8</code> energy:</p>
<pre><code>#############
#...........#
###A#B#C#D###
  #A#B#C#D#
  #########
</code></pre>
<p><em>What is the least energy required to organize the amphipods?</em></p>
</article>


In [2]:
from collections import Counter
from copy import deepcopy
from heapq import heappop, heappush
from itertools import chain
from typing import ClassVar

from more_itertools import minmax


@dataclass(frozen=True, order=True)
class State:
    """
    hallway:  0--1--2--3--4--5--6--7--8--9--10  ↓ i // 2 - 1 if i in {2.4.6.8}
                    |     |     |     |        	↑ 2*(i+1)
    rooms[0]:       0     1     2     3
                    |     |     |     |
    rooms[1]:       0     1     2     3
    """

    no_stopping: ClassVar[tuple[int, int, int, int]] = 2, 4, 6, 8
    destination_for: ClassVar[str] = "ABCD"
    energy_per_step: ClassVar[dict[str, int]] = {"A": 1, "B": 10, "C": 100, "D": 1_000}

    energy: int
    hallway: list[str]
    rooms: list[list[str]]

    def solve(self) -> int:
        heap = [self]
        seen = set()

        while heap:
            state = heappop(heap)

            if state.is_goal():
                return state.energy

            hs = tuple(
                tuple(r) for r in chain(state.hallway, state.rooms[0], state.rooms[1])
            )
            if hs in seen:
                continue

            seen.add(hs)

            state.move_from_hallway_to_room0(heap)
            state.move_from_room0_to_hallway(heap)
            state.move_from_room0_to_room1(heap)
            state.move_from_room1_to_room0(heap)
        return -1

    def is_goal(self) -> bool:
        return all(spot == "." for spot in self.hallway) and all(
            self.rooms[i][r] == self.destination_for[r]
            for i, r in product(range(len(self.rooms)), range(len(self.rooms[0])))
        )

    def move_from_hallway_to_room0(self, heap: list[State]) -> None:
        for i, occupant in enumerate(self.hallway):
            if occupant != ".":
                home_room = self.destination_for.index(occupant)
                if self.rooms[1][home_room] == self.rooms[0][home_room] == "." or (
                    self.rooms[1][home_room] == self.destination_for[home_room]
                    and self.rooms[0][home_room] == "."
                ):
                    to = 2 * (home_room + 1)
                    if to >= i:
                        current = i + 1
                        delta = 1
                    else:
                        current = i - 1
                        delta = -1

                    while (
                        0 <= current < len(self.hallway)
                        and self.hallway[current] == "."
                        and current != to
                    ):
                        current += delta

                    if current == to:
                        energy = (abs(to - i) + 1) * self.energy_per_step[occupant]
                        hallway = self.hallway[:]
                        rooms = deepcopy(self.rooms)
                        hallway[i], rooms[0][home_room] = (
                            rooms[0][home_room],
                            hallway[i],
                        )
                        energy = (
                            self.energy
                            + (abs(i - to) + 1) * self.energy_per_step[occupant]
                        )
                        heappush(heap, State(energy, hallway, rooms))

    def move_from_room0_to_room1(self, heap: list[State]) -> None:
        for i, occupant in enumerate(self.rooms[0]):
            if occupant == self.destination_for[i] and self.rooms[1][i] == ".":
                hallway = self.hallway[:]
                rooms = deepcopy(self.rooms)
                rooms[1][i], rooms[0][i] = rooms[0][i], rooms[1][i]
                energy = self.energy + self.energy_per_step[occupant]
                heappush(heap, State(energy, hallway, rooms))

    def move_from_room1_to_room0(self, heap: list[State]) -> None:
        for i, occupant in enumerate(self.rooms[1]):
            if (
                occupant not in (".", self.destination_for[i])
                and self.rooms[0][i] == "."
            ):
                hallway = self.hallway[:]
                rooms = deepcopy(self.rooms)
                rooms[1][i], rooms[0][i] = rooms[0][i], rooms[1][i]
                energy = self.energy + self.energy_per_step[occupant]
                heappush(heap, State(energy, hallway, rooms))

    def move_from_room0_to_hallway(self, heap: list[State]) -> None:
        for i, occupier in enumerate(self.rooms[0]):
            hallway_i = 2 * (i + 1)
            if occupier != "." and not (
                occupier == self.rooms[1][i] == self.destination_for[i]
            ):
                # move left
                steps = 2
                next_left = hallway_i - 1
                while next_left > -1 and self.hallway[next_left] == ".":
                    if next_left not in self.no_stopping:
                        hallway = self.hallway[:]
                        rooms = deepcopy(self.rooms)
                        hallway[next_left], rooms[0][i] = (
                            rooms[0][i],
                            hallway[next_left],
                        )
                        energy = self.energy + steps * self.energy_per_step[occupier]
                        heappush(heap, State(energy, hallway, rooms))
                    next_left -= 1
                    steps += 1

                # move right
                steps = 2
                next_right = hallway_i + 1
                while (
                    next_right < len(self.hallway) and self.hallway[next_right] == "."
                ):
                    if next_right not in self.no_stopping:
                        hallway = self.hallway[:]
                        rooms = deepcopy(self.rooms)
                        hallway[next_right], rooms[0][i] = (
                            rooms[0][i],
                            hallway[next_right],
                        )
                        energy = self.energy + steps * self.energy_per_step[occupier]
                        heappush(heap, State(energy, hallway, rooms))
                    next_right += 1
                    steps += 1

    def __str__(self) -> str:
        return "\n".join(
            (
                "#" * 13,
                f"#{''.join(spot for spot in self.hallway)}#",
                f"###{'#'.join(spot for spot in self.rooms[0])}###",
                f"  #{'#'.join(spot for spot in self.rooms[1])}#",
                f"  {'#'*9}",
            )
        )

    @classmethod
    def create_from_diagram(cls, diagram: str, energy: int = 0) -> State:
        data = list(re.sub(r"[^A-D^\.]", "", diagram))
        hallway = data[:11]
        rooms = [data[11:15], data[15:]]
        return State(energy, hallway, rooms)

In [3]:
solve_tests = [
    {
        "name": "initial is goal",
        "diagram": """
           #############
           #...........#
           ###A#B#C#D###
             #A#B#C#D#
             #########""",
        "expected": 0,
    },
    {
        "name": "penultimum from example",
        "diagram": """
          #############
          #.........A.#
          ###.#B#C#D###
            #A#B#C#D#
            ##########""",
        "expected": 8,
    },
    {
        "name": "2 before goal from example",
        "diagram": """
            #############
            #.....D.D.A.#
            ###.#B#C#.###
              #A#B#C#.#
              #########""",
        "expected": 7008,
    },
    {
        "name": "3 before goal from example",
        "diagram": """
            #############
            #.....D.....#
            ###.#B#C#D###
              #A#B#C#A#
              ######### 
            """,
        "expected": 9011,
    },
    {
        "name": "4 before goal from example",
        "diagram": """
            #############
            #.....D.....#
            ###B#.#C#D###
              #A#B#C#A#
              #########            """,
        "expected": 9051,
    },
    {
        "name": "5 before goal from example",
        "diagram": """
           #############
           #...B.......#
           ###B#.#C#D###
             #A#D#C#A#
             #########""",
        "expected": 12081,
    },
    {
        "name": "6 before goal from example",
        "diagram": """
            #############
            #...B.......#
            ###B#C#.#D###
              #A#D#C#A#
              #########""",
        "expected": 12481,
    },
    {
        "name": "example",
        "diagram": """
            #############
            #...........#
            ###B#C#B#D###
              #A#D#C#A#
              #########""",
        "expected": 12521,
    },
]


@test(tests=solve_tests)
def solve_test(diagram: str) -> bool:
    state = State.create_from_diagram(diagram)
    return state.solve()


Test initial is goal passed, for solve_test.
Test penultimum from example passed, for solve_test.
Test 2 before goal from example passed, for solve_test.
Test 3 before goal from example passed, for solve_test.
Test 4 before goal from example passed, for solve_test.
Test 5 before goal from example passed, for solve_test.
Test 6 before goal from example passed, for solve_test.
Test example passed, for solve_test.
Success


In [4]:
is_goal_tests = [
    {
        "name": "is goal",
        "diagram": """
           #############
           #...........#
           ###A#B#C#D###
             #A#B#C#D#
             #########""",
        "expected": True,
    },
    {
        "name": "is not goal",
        "diagram": """
           #############
           #...........#
           ###A#B#C#D###
             #A#B#D#C#
             #########""",
        "expected": False,
    },
    {
        "name": "Is not goal",
        "diagram": """
           #############
           #.....D.....#
           ###.#.#C#.###
             #A#B#C#D#
             #########""",
        "expected": False,
    },
]


@test(tests=is_goal_tests)
def is_goal_testzs(diagram: str) -> bool:
    state = State.create_from_diagram(diagram)
    return state.is_goal()


Test is goal passed, for is_goal_testzs.
Test is not goal passed, for is_goal_testzs.
Test Is not goal passed, for is_goal_testzs.
Success


In [5]:
def assert_function(actual: list[State], expected: list[str]) -> bool:
    if len(actual) != len(expected):
        return False

    for a, e in zip(
        sorted(actual), sorted(State.create_from_diagram(d, c) for d, c in expected)
    ):
        if a != e:
            print("**Error:")
            print(a)
            print(a.energy)
            print("\nshould be:\n")
            print(e)
            print(e.energy)
            return False
    return True

In [6]:
move_from_hallway_to_room0_tests = [
    {
        "name": "Move none from hallway to room[1] test 1",
        "diagram": """
           #############
           #.....D.....#
           ###.#.#C#D###
             #A#B#C#A#
             #########""",
        "expected": [],
    },
    {
        "name": "Move one from room[0] to room[1] test 2",
        "diagram": """
           #############
           #.....D.....#
           ###.#.#C#.###
             #A#B#C#D#
             #########""",
        "expected": [
            (
                """
               #############
               #...........#
               ###.#.#C#D###
                 #A#B#C#D#
                 #########""",
                4 * State.energy_per_step["D"],
            ),
        ],
    },
    {
        "name": "Move one from room[0] to room[1] test 3",
        "diagram": """
           #############
           #.....A.....#
           ###.#.#C#.###
             #.#B#C#D#
             #########""",
        "expected": [
            (
                """
               #############
               #...........#
               ###A#.#C#.###
                 #.#B#C#D#
                 #########""",
                4 * State.energy_per_step["A"],
            ),
        ],
    },
    {
        "name": "Move one from room[0] to room[1] test 4",
        "diagram": """
           #############
           #.....A.....#
           ###.#.#C#.###
             #B#B#C#D#
             #########""",
        "expected": [],
    },
]


@test(tests=move_from_hallway_to_room0_tests, assert_funct=assert_function)
def move_from_hallway_to_room0_test(diagram: str) -> list[State]:
    state = State.create_from_diagram(diagram)
    heap = []
    state.move_from_hallway_to_room0(heap)
    return heap


Test Move none from hallway to room[1] test 1 passed, for move_from_hallway_to_room0_test.
Test Move one from room[0] to room[1] test 2 passed, for move_from_hallway_to_room0_test.
Test Move one from room[0] to room[1] test 3 passed, for move_from_hallway_to_room0_test.
Test Move one from room[0] to room[1] test 4 passed, for move_from_hallway_to_room0_test.
Success


In [7]:
move_from_room0_to_room1_tests = [
    {
        "name": "Move none from room[0] to room[1] test 1",
        "diagram": """
            #############
            #...........#
            ###B#C#B#.###
              #.#.#.#A#
              #########""",
        "expected": [],
    },
    {
        "name": "Move one from room[0] to room[1] test 1",
        "diagram": """
            #############
            #...........#
            ###.#C#.#D###
              #A#D#C#.#
              #########""",
        "expected": [
            (
                """
                #############
                #...........#
                ###.#C#.#.###
                  #A#D#C#D#
                  #########""",
                State.energy_per_step["D"],
            ),
        ],
    },
]


@test(tests=move_from_room0_to_room1_tests, assert_funct=assert_function)
def move_from_room0_to_room1_test(diagram: str) -> list[State]:
    state = State.create_from_diagram(diagram)
    heap = []
    state.move_from_room0_to_room1(heap)
    return heap


Test Move none from room[0] to room[1] test 1 passed, for move_from_room0_to_room1_test.
Test Move one from room[0] to room[1] test 1 passed, for move_from_room0_to_room1_test.
Success


In [8]:
move_from_room1_to_room0_tests = [
    {
        "name": "Move none from room[1] to room[0] test 1",
        "diagram": """
            #############
            #...........#
            ###B#C#B#D###
              #A#D#C#A#
              #########""",
        "expected": [],
    },
    {
        "name": "Move one from room[1] to room[0] test 1",
        "diagram": """
            #############
            #...........#
            ###.#C#.#.###
              #A#D#C#A#
              #########""",
        "expected": [
            (
                """
                #############
                #...........#
                ###.#C#.#A###
                  #A#D#C#.#
                  #########""",
                State.energy_per_step["A"],
            ),
        ],
    },
]


@test(tests=move_from_room1_to_room0_tests, assert_funct=assert_function)
def move_from_room1_to_room0_test(diagram: str) -> list[State]:
    state = State.create_from_diagram(diagram)
    heap = []
    state.move_from_room1_to_room0(heap)
    return heap


Test Move none from room[1] to room[0] test 1 passed, for move_from_room1_to_room0_test.
Test Move one from room[1] to room[0] test 1 passed, for move_from_room1_to_room0_test.
Success


In [9]:
move_from_room0_to_hallway_tests = [
    {
        "name": "move_from_room0_to_hallway on initial diagram test 1",
        "diagram": """
            #############
            #...........#
            ###B#.#.#.###
              #A#.#.#.#
              #########""",
        "expected": [
            # A room[0]
            (
                """
              #############
              #B..........#
              ###.#.#.#.###
                #A#.#.#.#
                #########""",
                3 * State.energy_per_step["B"],
            ),
            (
                """
              #############
              #.B.........#
              ###.#.#.#.###
                #A#.#.#.#
                #########""",
                2 * State.energy_per_step["B"],
            ),
            (
                """
              #############
              #...B.......#
              ###.#.#.#.###
                #A#.#.#.#
                #########""",
                2 * State.energy_per_step["B"],
            ),
            (
                """
              #############
              #.....B.....#
              ###.#.#.#.###
                #A#.#.#.#
                #########""",
                4 * State.energy_per_step["B"],
            ),
            (
                """
              #############
              #.......B...#
              ###.#.#.#.###
                #A#.#.#.#
                #########""",
                6 * State.energy_per_step["B"],
            ),
            (
                """
              #############
              #.........B.#
              ###.#.#.#.###
                #A#.#.#.#
                #########""",
                8 * State.energy_per_step["B"],
            ),
            (
                """
              #############
              #..........B#
              ###.#.#.#.###
                #A#.#.#.#
                #########""",
                9 * State.energy_per_step["B"],
            ),
        ],
    },
    {
        "name": "move_from_room0_to_hallway on initial diagram test 2",
        "diagram": """
            #############
            #...........#
            ###B#C#B#D###
              #A#D#C#A#
              #########""",
        "expected": [
            # A room[0]
            (
                """
              #############
              #B..........#
              ###.#C#B#D###
                #A#D#C#A#
                #########""",
                3 * State.energy_per_step["B"],
            ),
            (
                """
              #############
              #.B.........#
              ###.#C#B#D###
                #A#D#C#A#
                #########""",
                2 * State.energy_per_step["B"],
            ),
            (
                """
              #############
              #...B.......#
              ###.#C#B#D###
                #A#D#C#A#
                #########""",
                2 * State.energy_per_step["B"],
            ),
            (
                """
              #############
              #.....B.....#
              ###.#C#B#D###
                #A#D#C#A#
                #########""",
                4 * State.energy_per_step["B"],
            ),
            (
                """
            #############
            #.......B...#
            ###.#C#B#D###
              #A#D#C#A#
              #########""",
                6 * State.energy_per_step["B"],
            ),
            (
                """
            #############
            #.........B.#
            ###.#C#B#D###
              #A#D#C#A#
              #########""",
                8 * State.energy_per_step["B"],
            ),
            (
                """
            #############
            #..........B#
            ###.#C#B#D###
              #A#D#C#A#
              #########""",
                9 * State.energy_per_step["B"],
            ),
            # B room[0]
            (
                """
            #############
            #C..........#
            ###B#.#B#D###
              #A#D#C#A#
              #########""",
                5 * State.energy_per_step["C"],
            ),
            (
                """
            #############
            #.C.........#
            ###B#.#B#D###
              #A#D#C#A#
              #########""",
                4 * State.energy_per_step["C"],
            ),
            (
                """
            #############
            #...C.......#
            ###B#.#B#D###
              #A#D#C#A#
              #########""",
                2 * State.energy_per_step["C"],
            ),
            (
                """
            #############
            #.....C.....#
            ###B#.#B#D###
              #A#D#C#A#
              #########""",
                2 * State.energy_per_step["C"],
            ),
            (
                """
            #############
            #.......C...#
            ###B#.#B#D###
              #A#D#C#A#
              #########""",
                4 * State.energy_per_step["C"],
            ),
            (
                """
            #############
            #.........C.#
            ###B#.#B#D###
              #A#D#C#A#
              #########""",
                6 * State.energy_per_step["C"],
            ),
            (
                """
            #############
            #..........C#
            ###B#.#B#D###
              #A#D#C#A#
              #########""",
                7 * State.energy_per_step["C"],
            ),
            # C room[0]
            (
                """
            #############
            #B..........#
            ###B#C#.#D###
              #A#D#C#A#
              #########""",
                7 * State.energy_per_step["B"],
            ),
            (
                """
            #############
            #.B.........#
            ###B#C#.#D###
              #A#D#C#A#
              #########""",
                6 * State.energy_per_step["B"],
            ),
            (
                """
            #############
            #...B.......#
            ###B#C#.#D###
              #A#D#C#A#
              #########""",
                4 * State.energy_per_step["B"],
            ),
            (
                """
            #############
            #.....B.....#
            ###B#C#.#D###
              #A#D#C#A#
              #########""",
                2 * State.energy_per_step["B"],
            ),
            (
                """
            #############
            #.......B...#
            ###B#C#.#D###
              #A#D#C#A#
              #########""",
                2 * State.energy_per_step["B"],
            ),
            (
                """
            #############
            #.........B.#
            ###B#C#.#D###
              #A#D#C#A#
              #########""",
                4 * State.energy_per_step["B"],
            ),
            (
                """
            #############
            #..........B#
            ###B#C#.#D###
              #A#D#C#A#
              #########""",
                5 * State.energy_per_step["B"],
            ),
            # D room[0]
            (
                """
            #############
            #D..........#
            ###B#C#B#.###
              #A#D#C#A#
              #########""",
                9 * State.energy_per_step["D"],
            ),
            (
                """
            #############
            #.D.........#
            ###B#C#B#.###
              #A#D#C#A#
              #########""",
                8 * State.energy_per_step["D"],
            ),
            (
                """
            #############
            #...D.......#
            ###B#C#B#.###
              #A#D#C#A#
              #########""",
                6 * State.energy_per_step["D"],
            ),
            (
                """
            #############
            #.....D.....#
            ###B#C#B#.###
              #A#D#C#A#
              #########""",
                4 * State.energy_per_step["D"],
            ),
            (
                """
            #############
            #.......D...#
            ###B#C#B#.###
              #A#D#C#A#
              #########""",
                2 * State.energy_per_step["D"],
            ),
            (
                """
            #############
            #.........D.#
            ###B#C#B#.###
              #A#D#C#A#
              #########""",
                2 * State.energy_per_step["D"],
            ),
            (
                """
            #############
            #..........D#
            ###B#C#B#.###
              #A#D#C#A#
              #########""",
                3 * State.energy_per_step["D"],
            ),
        ],
    },
    {
        "name": "No moves from room[0] to hallway test 3",
        "diagram": """
            #############
            #.....D.D.A.#
            ###.#B#C#.###
              #A#B#C#.#
              #########""",
        "expected": [],
    },
    {
        "name": "Moves going to right are blocked from room[0] to hallway test 4",
        "diagram": """
            #############
            #.....D.....#
            ###.#A#C#.###
              #A#B#C#.#
              #########""",
        "expected": [
            (
                """
              #############
              #...A.D.....#
              ###.#.#C#.###
                #A#B#C#.#
                #########""",
                2 * State.energy_per_step["A"],
            ),
            (
                """
              #############
              #.A...D.....#
              ###.#.#C#.###
                #A#B#C#.#
                #########""",
                4 * State.energy_per_step["A"],
            ),
            (
                """
              #############
              #A....D.....#
              ###.#.#C#.###
                #A#B#C#.#
                #########""",
                5 * State.energy_per_step["A"],
            ),
        ],
    },
    {
        "name": "Moves going to left are blocked from room[0] to hallway test 5",
        "diagram": """
            #############
            #...D.......#
            ###.#A#C#.###
              #A#B#C#.#
              #########""",
        "expected": [
            (
                """
              #############
              #...D.A.....#
              ###.#.#C#.###
                #A#B#C#.#
                #########""",
                2 * State.energy_per_step["A"],
            ),
            (
                """
              #############
              #...D...A...#
              ###.#.#C#.###
                #A#B#C#.#
                #########""",
                4 * State.energy_per_step["A"],
            ),
            (
                """
              #############
              #...D.....A.#
              ###.#.#C#.###
                #A#B#C#.#
                #########""",
                6 * State.energy_per_step["A"],
            ),
            (
                """
              #############
              #...D......A#
              ###.#.#C#.###
                #A#B#C#.#
                #########""",
                7 * State.energy_per_step["A"],
            ),
        ],
    },
]


@test(tests=move_from_room0_to_hallway_tests, assert_funct=assert_function)
def move_from_room0_to_hallway_test(diagram: str) -> list[State]:
    state = State.create_from_diagram(diagram)
    heap = []
    state.move_from_room0_to_hallway(heap)
    return heap


Test move_from_room0_to_hallway on initial diagram test 1 passed, for move_from_room0_to_hallway_test.
Test move_from_room0_to_hallway on initial diagram test 2 passed, for move_from_room0_to_hallway_test.
Test No moves from room[0] to hallway test 3 passed, for move_from_room0_to_hallway_test.
Test Moves going to right are blocked from room[0] to hallway test 4 passed, for move_from_room0_to_hallway_test.
Test Moves going to left are blocked from room[0] to hallway test 5 passed, for move_from_room0_to_hallway_test.
Success


In [10]:
constructor_tests = [
    {
        "name": "Constructor test 1",
        "diagram": """
            #############
            #...........#
            ###B#C#B#D###
              #A#D#C#A#
              #########""",
        "expected": State(
            0,
            [".", ".", ".", ".", ".", ".", ".", ".", ".", ".", "."],
            [
                ["B", "C", "B", "D"],
                ["A", "D", "C", "A"],
            ],
        ),
    },
    {
        "name": "Constructor test 2",
        "diagram": """
            #############
            #.....D.D.A.#
            ###.#B#C#.###
              #A#B#C#.#
              #########""",
        "expected": State(
            0,
            [".", ".", ".", ".", ".", "D", ".", "D", ".", "A", "."],
            [
                [".", "B", "C", "."],
                ["A", "B", "C", "."],
            ],
        ),
    },
]


@test(tests=constructor_tests)
def constructor_test(diagram: str) -> State:
    return State.create_from_diagram(diagram)


Test Constructor test 1 passed, for constructor_test.
Test Constructor test 2 passed, for constructor_test.
Success


In [11]:
example = """
#############
#...........#
###B#C#B#D###
  #A#D#C#A#      
  #########    
"""

assert State.create_from_diagram(example).solve() == 12521

In [12]:
puzzle = """
#############
#...........#
###D#D#C#C###
  #B#A#B#A#  
  #########  
"""
print(f"Part I: { State.create_from_diagram(puzzle).solve() }")

Part I: 16059


<link href="style.css" rel="stylesheet"></link>
<main>

<p>Your puzzle answer was <code>16059</code>.</p><p class="day-success">The first half of this puzzle is complete! It provides one gold star: *</p>


<article class="day-desc"><h2 id="part2">--- Part Two ---</h2><p>As you prepare to give the amphipods your solution, you notice that the diagram they handed you was actually folded up. As you unfold it, you discover an extra part of the diagram.</p>
<p>Between the first and second lines of text that contain amphipod starting positions, insert the following lines:</p>
<pre><code>  #D#C#B#A#
  #D#B#A#C#
</code></pre>
<p>So, the above example now becomes:</p>
<pre><code>#############
#...........#
###B#C#B#D###
  <em>#D#C#B#A#
  #D#B#A#C#</em>
  #A#D#C#A#
  #########
</code></pre>
<p>The amphipods still want to be organized into rooms similar to before:</p>
<pre><code>#############
#...........#
###A#B#C#D###
  #A#B#C#D#
  #A#B#C#D#
  #A#B#C#D#
  #########
</code></pre>
<p>In this updated example, the least energy required to organize these amphipods is <code><em>44169</em></code>:</p>
<pre><code>#############
#...........#
###B#C#B#D###
  #D#C#B#A#
  #D#B#A#C#
  #A#D#C#A#
  #########

#############
#..........D#
###B#C#B#.###
#D#C#B#A#
#D#B#A#C#
#A#D#C#A#
#########

#############
#A.........D#
###B#C#B#.###
#D#C#B#.#
#D#B#A#C#
#A#D#C#A#
#########

#############
#A........BD#
###B#C#.#.###
#D#C#B#.#
#D#B#A#C#
#A#D#C#A#
#########

#############
#A......B.BD#
###B#C#.#.###
#D#C#.#.#
#D#B#A#C#
#A#D#C#A#
#########

#############
#AA.....B.BD#
###B#C#.#.###
#D#C#.#.#
#D#B#.#C#
#A#D#C#A#
#########

#############
#AA.....B.BD#
###B#.#.#.###
#D#C#.#.#
#D#B#C#C#
#A#D#C#A#
#########

#############
#AA.....B.BD#
###B#.#.#.###
#D#.#C#.#
#D#B#C#C#
#A#D#C#A#
#########

#############
#AA...B.B.BD#
###B#.#.#.###
#D#.#C#.#
#D#.#C#C#
#A#D#C#A#
#########

#############
#AA.D.B.B.BD#
###B#.#.#.###
#D#.#C#.#
#D#.#C#C#
#A#.#C#A#
#########

#############
#AA.D...B.BD#
###B#.#.#.###
#D#.#C#.#
#D#.#C#C#
#A#B#C#A#
#########

#############
#AA.D.....BD#
###B#.#.#.###
#D#.#C#.#
#D#B#C#C#
#A#B#C#A#
#########

#############
#AA.D......D#
###B#.#.#.###
#D#B#C#.#
#D#B#C#C#
#A#B#C#A#
#########

#############
#AA.D......D#
###B#.#C#.###
#D#B#C#.#
#D#B#C#.#
#A#B#C#A#
#########

#############
#AA.D.....AD#
###B#.#C#.###
#D#B#C#.#
#D#B#C#.#
#A#B#C#.#
#########

#############
#AA.......AD#
###B#.#C#.###
#D#B#C#.#
#D#B#C#.#
#A#B#C#D#
#########

#############
#AA.......AD#
###.#B#C#.###
#D#B#C#.#
#D#B#C#.#
#A#B#C#D#
#########

#############
#AA.......AD#
###.#B#C#.###
#.#B#C#.#
#D#B#C#D#
#A#B#C#D#
#########

#############
#AA.D.....AD#
###.#B#C#.###
#.#B#C#.#
#.#B#C#D#
#A#B#C#D#
#########

#############
#A..D.....AD#
###.#B#C#.###
#.#B#C#.#
#A#B#C#D#
#A#B#C#D#
#########

#############
#...D.....AD#
###.#B#C#.###
#A#B#C#.#
#A#B#C#D#
#A#B#C#D#
#########

#############
#.........AD#
###.#B#C#.###
#A#B#C#D#
#A#B#C#D#
#A#B#C#D#
#########

#############
#..........D#
###A#B#C#.###
#A#B#C#D#
#A#B#C#D#
#A#B#C#D#
#########

#############
#...........#
###A#B#C#D###
#A#B#C#D#
#A#B#C#D#
#A#B#C#D#
#########
</code></pre>

<p>Using the initial configuration from the full diagram, <em>what is the least energy required to organize the amphipods?</em></p>
</article>


<link href="style.css" rel="stylesheet"></link>


In [13]:
# print(f"Part II: { Toboggan(puzzle).count_tree_on_path_down_on_slopes()}")

<link href="style.css" rel="stylesheet"></link>
<main>

<p>Your puzzle answer was <code>3093068400</code>.</p><p class="day-success">Both parts of this puzzle are complete! They provide two gold stars: **</p>

</main>
